In [1]:
import pandas as pd
from nltk.corpus import stopwords
import re

import warnings
warnings.filterwarnings('ignore')

Модели для обработки текста: <br>
@ChatGPT_BotNews, @chatgpt_tgm_bot <br>
ChatGPT 4.0

Запрос в ChatGPT: <br>
Find adverse drug reaction (or reactions) code(s) in the sentences below. Make the result as such (id number + code, for example: 1 C0684249)

Выход: <br>
... <br>
262: C0005801  
263: C0030146  
264: C0013404, C0020538, C0238853, C0236442  
265: C0011991, C0018017, C0025385, C0159077, C0024256, C0040634 <br>
...

In [2]:
# сохраняем выход в ADR_GPT.txt

In [3]:
adr = pd.read_csv('ADR_GPT.txt', sep=':')
adr.head()

,id,code
0,1,"C0018402, C0025266, C0024530"
1,2,C0036690
2,3,C0036690
3,4,C2931847
4,5,C2931847


In [4]:
# индекс и айди разные потому что в "побочные_эффекты" были пропущены некоторые айдишники жалоб (например: 1182, 1184)

In [5]:
def outliers(a):
    if a[0] == 'C':   # ищем все то что не начинается с С____ (то есть не содержит ADR коды)
        return 'NaN'  # но это будет не пандасовский флоат НаН а просто строковый формат
    else:
        return a
    
adr['code'] = adr['code'].apply(lambda x: outliers(x))

In [6]:
adr.loc[adr['code'] == 'NaN']

,id,code
600,565,NaN
601,566,NaN
602,567,NaN
603,568,NaN
649,614,NaN
650,615,NaN
651,616,NaN
652,617,NaN
653,618,NaN
654,619,NaN


Ну в принципе 14 пропусков это норм, тем более что это могло произойти из-за залипания gpt (слишком много токенов и пропускает логическую последовательность). Проверим это

In [7]:
complaints = pd.read_csv('побочные_эффекты.txt', delimiter='\t')
complaints.head()

,id,Text
0,1,"extreme weight gain, short-term memory loss, h..."
1,2,COMPLETELY DESTROYED SEXUALLY FUNCTIONING .
2,3,Just TWO tablets of Lexapro 10mg completely de...
3,4,It's called PSSD: post-SSRI sexual dysfunction.
4,5,And there is a chance that it will give you PS...


In [8]:
pd.set_option('display.max_colwidth', None)

adr_full = pd.merge(complaints, adr, on=['id'])
adr_full.head()

,id,Text,code
0,1,"extreme weight gain, short-term memory loss, hair loss.","C0018402, C0025266, C0024530"
1,2,COMPLETELY DESTROYED SEXUALLY FUNCTIONING .,C0036690
2,3,"Just TWO tablets of Lexapro 10mg completely destroyed my sexual functioning, probably for life.",C0036690
3,4,It's called PSSD: post-SSRI sexual dysfunction.,C2931847
4,5,"And there is a chance that it will give you PSSD, which as the name suggests persists even after you stop taking the drug, Just google 'PSSD' and you'll see what I mean, So please: NEVER take this drug, not even one tablet.",C2931847


In [9]:
# это я подгоняла текст для поиску по словарю но потом вспомнила что гпт сам предобрабатывает так что это можно и не делать
adr_full['Text'] = adr_full['Text'].str.lower()
adr_full['Text'] = adr_full['Text'].str.split(',')
adr_full['Text'] = adr_full['Text'].apply(lambda x: [item for item in x if str(item) not in stopwords.words('english')])
adr_full['Text'] = adr_full['Text'].apply(lambda x: ''.join(x))

adr_full.head()

,id,Text,code
0,1,extreme weight gain short-term memory loss hair loss.,"C0018402, C0025266, C0024530"
1,2,completely destroyed sexually functioning .,C0036690
2,3,just two tablets of lexapro 10mg completely destroyed my sexual functioning probably for life.,C0036690
3,4,it's called pssd: post-ssri sexual dysfunction.,C2931847
4,5,and there is a chance that it will give you pssd which as the name suggests persists even after you stop taking the drug just google 'pssd' and you'll see what i mean so please: never take this drug not even one tablet.,C2931847


In [11]:
adr_full.loc[adr_full['code'] == 'NaN', ['id', 'Text']]

,id,Text
600,565,i can deal with all these side effects but would love to get rid of the fatigue.
601,566,sometimes i think the vivid dreaming keeps you from getting quality sleep however this medicine has been great-i had anxiety from divorce new home new job and dealing with a severely depressed ex-spouse (who is now on effexor).
602,567,some typical antidepressant side effects but nothing too serious.
603,568,i had a few emotional side effects but they went away very quickly.
649,614,stayed away from my grandkids cuz i didn't want them to find me dead.
650,615,lost all of my energy.
651,616,developed shortness of breath fibromyalgia type pain arthritis pain.
652,617,could barely walk.
653,618,i have been off work for 8 months.
654,619,i lost my job and will have to find another one.


In [12]:
len(adr_full.loc[adr_full['code'] == 'NaN', ['id', 'Text']])

14

Ещё раз закинем в gpt

Выход:<br><br>
565: C0005688  
566: C0005689  
567: C0018792  
568: C1963712  
614: C0015967  
615: C0747802  
616: C0000737, C0015676, C0003862  
617: C0340556  
618: C0038431  
619: C0041948, C0231269  
620: C0018671, C0013390, C0013405, C0021051, C0156969  
823: C0043096  
877: C0340556  
999: C0027497 <br><br>
Да, gpt нашел коды, забиваем их в таблицу

In [13]:
additional = {}
with open('additional.txt','r') as inf:
    for line in inf:
        line = re.sub(' ', '', line)
        code = re.sub('\n', '', line.split(':')[1])
        additional[line.split(':')[0]] = code

additional

{'565': 'C0005688',
 '566': 'C0005689',
 '567': 'C0018792',
 '568': 'C1963712',
 '614': 'C0015967',
 '615': 'C0747802',
 '616': 'C0000737,C0015676,C0003862',
 '617': 'C0340556',
 '618': 'C0038431',
 '619': 'C0041948,C0231269',
 '620': 'C0018671,C0013390,C0013405,C0021051,C0156969',
 '823': 'C0043096',
 '877': 'C0340556',
 '999': 'C0027497'}

In [14]:
for key in additional:
    ind = adr_full.loc[adr_full['id'] == int(key)].index.tolist() # находим табличный индекс айдишнику
    adr_full['code'][ind[0]] = additional[key]

In [15]:
adr_full.loc[adr_full['code'] == 'NaN', ['id', 'Text']]

,id,Text


=> пропусков нет

## Вариант только с айди жалобы и кодами (формат str)

"tychina_3.csv"

In [16]:
cols = ['id', 'code']
adr_dz = adr_full[cols]
adr_dz.head()

,id,code
0,1,"C0018402, C0025266, C0024530"
1,2,C0036690
2,3,C0036690
3,4,C2931847
4,5,C2931847


In [17]:
adr_dz.to_csv('tychina_3.csv')

## Вариант как в домашнем задании

Оставляю оба варианта тк в моем решении был только 1 уровень (не 5), но под формат подгоню

"tychina_3_2.csv"

In [18]:
for el in range(1, 5):
    col_name = 'level_' + str(el)
    adr_dz[col_name] = 0

adr_dz['level_5'] = 1
adr_dz.head()

,id,code,level_1,level_2,level_3,level_4,level_5
0,1,"C0018402, C0025266, C0024530",0,0,0,0,1
1,2,C0036690,0,0,0,0,1
2,3,C0036690,0,0,0,0,1
3,4,C2931847,0,0,0,0,1
4,5,C2931847,0,0,0,0,1


In [19]:
adr_dz.to_csv('tychina_3_2.csv') # не убирала индексы!!